In [17]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

In [18]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [21]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')
def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords

In [20]:
# dset_path = Path('/media/brennan/Data/ml/gazecapture/')
dset_path = Path('../gazecapture/')

# Export to hdf5
Not Used - See below for better soln.

In [9]:
# Extracting and storing X values
def extract_photo(f, case): return get_frame(case, f)

def extract(case):
    F = frame_data(case) # list of file names correlated to camera images 
    with Pool(6) as p:  # multi core usale
        x = p.map(functools.partial(extract_photo, case=case), F)
    return x

def extract_to_hdf5(cname, bs, extract_fcn): # bs dsets per hp5y file
    f = h5py.File('{}-0.hdf5'.format(cname), 'w') 
    for i, case in enumerate(list(dset_path.iterdir())):
        cn = case.name
        edata = extract_fcn(case)
        
        dset = f.create_dataset('dset'+cn, (len(edata), 224, 224, 3))
        dset[...] = edata

        if i % bs == 0 and i!= 0:
            f.close()
            f = h5py.File('{}-{}.hdf5'.format(cname, i//bs), 'w')

In [2]:
extract_to_hdf5('X', 500, extract)

# Extract File Names For Keras Generator

In [35]:
# split into train / test sets
# then split the train set into train / val sets
fnames = []
XCam = []
YCam = []

i = 0
for case in tqdm(list(dset_path.iterdir())):
    F = frame_data(case)
    xcam, ycam = coordinate_data(case)
    XCam += xcam
    YCam += ycam
    if len(F) != len(xcam): print(case)
        
    for f in F:
        fnames.append('{}/frames/{}'.format(case.name, f))
        
    if i > 100:
        break
    i += 1

  6%|▌         | 76/1290 [00:00<00:03, 370.72it/s]


In [36]:
len(XCam), len(fnames)

(156527, 156527)

In [38]:
# create dataframe from data
df = pd.DataFrame(data={'file_names': fnames, 'XCam': XCam,
                       'YCam': YCam})
df.head()

,file_names,XCam,YCam
0,01128/frames/00000.jpg,5.133576,-11.228056
1,01128/frames/00001.jpg,5.133576,-11.228056
2,01128/frames/00002.jpg,5.133576,-11.228056
3,01128/frames/00003.jpg,-5.438229,-1.499688
4,01128/frames/00004.jpg,-5.438229,-1.499688


In [39]:
test_size = 0.1
train, test = train_test_split(df, test_size=test_size)

In [40]:
train.shape, test.shape

((140874, 3), (15653, 3))

In [41]:
train.to_csv('train-df.csv')

In [42]:
test.to_csv('test-df.csv')